## Connect DB

In [20]:
import pandas as pd  
from sqlalchemy import create_engine  
import cx_Oracle
import datetime
import re
from cnsenti import Sentiment
from datetime import date, timedelta

In [21]:
def read_sql(name_list, time):
    dataDf = pd.DataFrame(data=None, index=None, columns=['ID', 'COMP_NM', 'NEWS_TITLE', 'NEWS_TIME', 'NEWS_SOURCE', 'NEWS_CONTENT', 'THE_URL'], dtype=None, copy=False)
    
    #进行oracle服务器设置，用户名；密码；HOST数据库IP地址；PORT端口号；SERVICE_NAME
    db=cx_Oracle.connect('tyyuqing','Tyyuq*09','172.22.5.77:1521/dcdw2')  
    cr=db.cursor()  #数据库游标

    if len(time) == 0 or len(time) == 1:
        if len(time) == 0:
            day = date.today()-timedelta(days=1)
            day = day.strftime('%Y-%m-%d')
        else:
            day = time[0]
        for name in name_list:
            # 此sql要求数据库表中的NEWS_TIME字段必须是yyyy-mm-dd形式
            sql="""select * from CLEAN_CCTV where COMP_NM=:n and to_char(NEWS_TIME, 'yyyy-mm-dd')=:t""" ## 此处写列表页和详情页的left jion sql
            #print(sql)
            cr.execute(sql, n = name, t = day)  #执行查询语句

            data=cr.fetchall()  #返回多个记录(rows)：fetchone()返回单个的元组，也就是一条记录(row)
            #data.columns=index[0]  #更新列名
            index = pd.DataFrame(cr.description)#提取列名
            dataDf = dataDf.append(pd.DataFrame(data,columns=index[0]), ignore_index = True)
        
    if len(time) == 2:
        start, end = time[0], time[1]
        
        for name in name_list:
            #print(name)
            sql="""select * from CLEAN_CCTV where COMP_NM=:n and NEWS_TIME between to_date(:s,'yyyy-MM-dd') AND to_date(:e,'yyyy-MM-dd')""" ## 此处写列表页和详情页的left jion sql
            #print(sql)
            cr.execute(sql, n = name, s = start, e = end)  #执行查询语句

            data=cr.fetchall()  #返回多个记录(rows)：fetchone()返回单个的元组，也就是一条记录(row)
            #data.columns=index[0]  #更新列名
            index = pd.DataFrame(cr.description)#提取列名
            dataDf = dataDf.append(pd.DataFrame(data,columns=index[0]), ignore_index = True)
        
    return dataDf

def clean_data(data):
# 去除公司名中可能包含的空格
    for i in range(len(data)):
        data.loc[i,'COMP_NM'] = re.sub(r"\s+", "", data.loc[i,'COMP_NM'])

# 修改日期格式为yyyy-mm-dd
#     for i in range(len(data)):
#         data.loc[i, 'NEWS_TIME'] = str(data.loc[i, 'NEWS_TIME']).split('日')[0]
#     for i in range(len(data)):
#         data.loc[i, 'NEWS_TIME'] = str(data.loc[i, 'NEWS_TIME']).replace('年', '-').replace('月', '-')
# 清除无效数据
    for i in range(len(data)):
        if (str(data.loc[i, 'COMP_NM']) in str(data.loc[i, 'NEWS_TITLE'])) or (str(data.loc[i, 'COMP_NM']) in str(data.loc[i, 'NEWS_CONTENT'])):
            data.drop(data.index[i])

    data = data.dropna()
    data = data.drop_duplicates(['COMP_NM', 'NEWS_TITLE'], keep='first').drop_duplicates(['COMP_NM', 'NEWS_CONTENT'], keep='first')
    data = data.reset_index(drop=True)
    data = data.drop(['ID'], axis=1)
    return data

def tagging(data, pos_path, neg_path):
    for i in range(len(data)):
        senti = Sentiment(pos=pos_path,  #正面词典txt文件相对路径
                          neg=neg_path,  #负面词典txt文件相对路径
                          merge=True,             #融合cnsenti自带词典和用户导入的自定义词典
                          encoding='utf-8')      #两txt均为utf-8编码

        test_text=str(data.loc[i,'NEWS_CONTENT'])
        result = senti.sentiment_calculate(test_text)

        if result['neg']>result['pos']:
            data.loc[i, 'TAGS'] = 'negative'
        elif result['neg']<result['pos']:
            data.loc[i, 'TAGS'] = 'positive'
        else: 
            data.loc[i, 'TAGS'] = 'neutral'

    dic = {}
    for i in range(len(data)):
        if data.loc[i, 'COMP_NM'] in dic:
            dic[data.loc[i, 'COMP_NM']].append((data.loc[i, 'THE_URL'], data.loc[i, 'TAGS']))
        else:
            dic[data.loc[i, 'COMP_NM']] = [(data.loc[i, 'THE_URL'], data.loc[i, 'TAGS'])]
    print(dic)
    return dic

def main():
    data = read_sql(['太平洋', '老百姓', '上汽集团', '比亚迪'], ['2020-09-10', '2020-12-10'])
    data = clean_data(data)
    tagging(data, 'dict/pos.txt', 'dict/neg.txt')    

In [5]:
%%time
if __name__ == '__main__':
    main()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\JxmDe\AppData\Local\Temp\jieba.cache
Loading model cost 1.450 seconds.
Prefix dict has been built successfully.


{'太平洋': [('https://news.cctv.com/2020/12/05/ARTIunSiUN6XjWadG9LYDyVQ201205.shtml', 'positive'), ('https://news.cctv.com/2020/11/20/ARTIdpLrBgXOCutstNbSLMkR201120.shtml', 'positive'), ('https://news.cctv.com/2020/10/25/ARTIS8dP0hpxKuFRHrJD01TV201025.shtml', 'negative'), ('https://news.cctv.com/2020/11/19/ARTIPiauY7I7Dy0wRTVT8Yct201119.shtml', 'positive'), ('https://news.cctv.com/2020/11/20/ARTIOv6TtM5igaT6Wkx4kppT201120.shtml', 'positive'), ('https://news.cctv.com/2020/11/28/ARTIu92k3fU4RQxeadeXYSEk201128.shtml', 'positive'), ('https://news.cctv.com/2020/11/12/ARTIlZw1Wj5eNe0jBxJTpPU1201112.shtml', 'positive'), ('https://news.cctv.com/2020/11/25/ARTIbPUKf7wkGkz7amr4e9UC201125.shtml', 'positive'), ('https://news.cctv.com/2020/11/20/ARTItvtq0gVoRnrOdr9zfSBR201120.shtml', 'positive'), ('https://news.cctv.com/2020/12/01/ARTIx3fWooRQM1MoLCgVqAwV201201.shtml', 'positive'), ('https://news.cctv.com/2020/12/03/ARTIGOBSo9oWvcdxzZIwLcBT201203.shtml', 'positive'), ('https://news.cctv.com/2020/11/09

## Categorize

In [70]:
data=pd.read_csv('raw data/clean_xinhuanet.csv')[:10]

In [74]:
categorize(data)

,ID,COMP_NM,NEWS_TITLE,NEWS_TIME,NEWS_SOURCE,NEWS_CONTENT,THE_URL,SEN,Category
0,1,天津港,【2020·指尖城市】天津港：用技术打造智慧港口新标杆 用智慧赋能港口建设新海蓝,2020-10-28 16:33:16,新华网,新华网天津10月27日电（宫碧莹）2020年，位于渤海湾上海河入海口的天津港建设愈发令人...,http://www.xinhuanet.com/info/2020-10/28/c_139...,新华网天津10月27日电（宫碧莹）2020年，位于渤海湾上海河入海口的天津港建设愈发令人...,环保问题.
1,3,天津港,天津港再添“一带一路”新航线,2020-10-28 10:48:46,新华网,新华社天津10月28日电（记者王井怀）日前，随着“金星佐伊”轮靠泊天津港联盟国际集装箱码...,http://www.xinhuanet.com/fortune/2020-10/28/c_...,新华社天津10月28日电（记者王井怀）日前，随着“金星佐伊”轮靠泊天津港联盟国际集装箱码...,生产产能.
2,6,中国科技出版传媒股份有限公司,镜观中国·新华社国内新闻照片一周精选,2020-11-28 19:20:43,新华网,雪后长城\r\n 11月21日，北京出现降雪，雪后的慕田峪长城银装素裹，云雾缭绕，美如...,http://www.xinhuanet.com/2020-11/28/c_11267981...,,其他
3,7,中国科技出版传媒股份有限公司,金典“夺冠”，安慕希“售罄”！伊利电商双11稳居行业榜首,2020-11-12 16:51:49,新华网,今年双11期间，乳业表现尤为活跃，成为电商销售最火热的品类之一。星图数据显示，11月11...,http://www.xinhuanet.com/food/2020-11/12/c_112...,,其他
4,8,中国科技出版传媒股份有限公司,第三十届中国新闻奖获奖作品目录（348件）,2020-11-02 23:52:17,新华网,新华社北京11月2日电\r\n第三十届中国新闻奖获奖作品目录（348件）\r\n 特别...,http://www.xinhuanet.com/2020-11/02/c_11266892...,,其他
5,10,万华化学,从“华山一条路”走出的“一体化”化学王国——新时代新国企万华改革创新启示录（中）,2019-08-13 10:52:14,经济参考报,万华烟台工业园装置图。资料照片\r\n\r\n 在烟台万华化学集团中央研究院，工作人员在做...,http://www.xinhuanet.com/2019-08/13/c_11248692...,资料照片\r\n\r\n 在烟台万华化学集团中央研究院，工作人员在做聚丙烯材料摆锤冲击试验...,生产产能.产品信息.市场相关.专利.
6,12,新世界,“云”上生活——从世界VR产业大会云峰会品味生活新场景,2020-10-19 20:18:16,新华网,新华社南昌10月19日电（记者郭杰文）19日，2020世界VR产业大会云峰会在江西南昌开...,http://www.xinhuanet.com/2020-10/19/c_11266311...,,其他
7,14,新世界,科普：量子是什么？从“微观世界规律”到人类“新物理革命”,2020-10-19 20:13:15,新华网,新华社合肥10月19日电 题：科普：量子是什么？从“微观世界规律”到人类“新物理革命”\...,http://www.xinhuanet.com/2020-10/19/c_11266311...,,其他
8,17,新世界,新华时评：中国经济正增长为世界注入正能量,2020-10-19 18:04:50,新华网,新华社北京10月19日电 题：中国经济正增长为世界注入正能量\r\n 新华社记者刘红霞...,http://www.xinhuanet.com/fortune/2020-10/19/c_...,,其他
9,19,罗平锌电,被点名通报后 云南罗平锌电已投入1.9亿整改环境隐患,2019-02-21 18:31:42,新华网,新华社昆明2月21日电（记者 白靖利）“再过几个月，这里将变成一个美丽的运动场。”面对现...,http://www.xinhuanet.com/local/2019-02/21/c_11...,”面对现场二十余名媒体记者，罗平锌电股份有限公司副董事长李尤立说。\r\n 2018年6月...,环保问题.警示.管理相关.生产产能.


In [86]:
import re
import pandas as pd

def categorize(data,path='dict/category.xlsx'):
    # function to cut the sentence
    def cut_sen(text):
        pattern = r'\|\?|!|。|！|…|？| '
        result_list = re.split(pattern, text)
        return result_list[:-1]
    
    # store relevant sentences
    for i in range(len(data)):
        sentence=cut_sen(data.loc[i,'NEWS_CONTENT'])
        res=[]
        # loop through every sentence
        for j in sentence:
            if data.loc[i,'COMP_NM'] in j:
                res.append(j)
        data.loc[i,'SEN']='。'.join(res)
    
    # create category dic
    cat=pd.read_excel('dict/category.xlsx')
    for i in range(len(key)):
        d[key[i]]=str(value[i]).split('，')
        
    # append tags
    for i in range(len(data)):
        data.loc[i, 'Category'] = ''

        for k in key:
            for v in d[k]:
                if v in data.loc[i, 'SEN']: 
                    data.loc[i, 'Category']+=k
                    data.loc[i, 'Category']+='.'

        if len(data.loc[i, 'Category'])==0:
            data.loc[i, 'Category']='其他'
            
    return data

In [38]:
# cut sentences
import re
def cut_sen(text):
    pattern = r'\|\?|!|。|！|…|？'
    result_list = re.split(pattern, text)
    return result_list[:-1]

cut_sen('由于舆情数据的特殊!性以及其具有极高的复杂度我们的数据中仍然包含少量的无关新闻，倘若这部分无关新闻被判定为负向新闻并推送给相关企业，则会大大损害我们产品的口碑。因此，要根据不同舆情$数据的特点，探究更加准确的数据清洗/筛选规则，这将是后续工作中一个值得重点研究的方向。')

['由于舆情数据的特殊',
 '性以及其具有极高的复杂度我们的数据中仍然包含少量的无关新闻，倘若这部分无关新闻被判定为负向新闻并推送给相关企业，则会大大损害我们产品的口碑',
 '因此，要根据不同舆情$数据的特点，探究更加准确的数据清洗/筛选规则，这将是后续工作中一个值得重点研究的方向']

In [75]:
import pandas as pd
data=pd.read_csv('raw data/clean_eeo.csv')[:10]

In [76]:
data.head()

,ID,COMP_NM,NEWS_TITLE,NEWS_TIME,NEWS_SOURCE,NEWS_CONTENT,THE_URL
0,1,西部证券,时隔两年再放行 证监会批准纽银梅隆西部基金公司成立,2010-07-08 08:07,经济观察网,经济观察网 记者 欧阳晓红 时隔两年，中国基金市场终添一新员。7月7日，本网获悉，中国证券监...,http://www.eeo.com.cn/2010/0708/174886.shtml
1,2,西部证券,西部证券IPO搁浅,2010-03-08 07:03,经济观察网,经济观察报 记者 中彬 李保华 西部证券上市申请的靴子终于落地。\n\r\n\r\n本报独家...,http://www.eeo.com.cn/2010/0308/164497.shtml
2,3,拉卡拉,产业互联网战略成果显现 拉卡拉前三季度营收净利双增长,2020-10-22 19:58,经济观察网,10月22日，拉卡拉发布2020年三季报，数据显示，第三季度营收16.14亿元，同比增长35...,http://www.eeo.com.cn/2020/1022/424674.shtml
3,4,拉卡拉,拉卡拉受邀亮相2020中国国际服务贸易交易会,2020-09-05 12:43,经济观察网,9月5日，2020年中国国际服务贸易交易会（以下简称“服贸会”）在中国国家会议中心（北京）举...,http://www.eeo.com.cn/2020/0905/408736.shtml
4,5,拉卡拉,拉卡拉成功上市 从支付收单到全方位赋能商户 上市之后的拉卡拉 会给市场带来些什么？,2019-04-26 19:23,经济观察网,4月25日，头顶A股“支付第一股”光环的拉卡拉正式登陆深交所创业板，其后续的表现无疑将成为第...,http://www.eeo.com.cn/2019/0426/354516.shtml


In [88]:
df=categorize(data)
df[['COMP_NM','SEN','Category']]
# df[['NEWS_CONTENT','SEN']]

,COMP_NM,SEN,Category
0,西部证券,7月7日，本网获悉，中国证券监督管理委员会（“证监会”）正式批准纽约梅隆和西部证券在中国合资...,管理相关.投融资.
1,西部证券,西部证券上市申请的靴子终于落地。\n\r\n\r\n本报独家获悉，西部证券因不符合券商上市新...,减资/分立/合并.管理相关.股权激励.经营业绩.投融资.新股发行.上市退市.政策影响.
2,拉卡拉,10月22日，拉卡拉发布2020年三季报，数据显示，第三季度营收16.14亿元，同比增长35...,经营业绩.产品信息.市场相关.上市退市.评级信息.
3,拉卡拉,作为金融科技领先企业、第三方支付头部机构，拉卡拉受邀参加了此次服贸会的金融服务专题展，与15...,管理相关.经营相关.产品信息.牌照.
4,拉卡拉,4月25日，头顶A股“支付第一股”光环的拉卡拉正式登陆深交所创业板，其后续的表现无疑将成为第...,破产清算.管理相关.经营相关.产品信息.项目签约.市场相关.上市退市.
5,拉卡拉,几经波折之后，拉卡拉离资本市场终于尚存几步之遥。\r\n3月12日，证监会官网披露了拉卡拉支...,经营相关.产品信息.市场相关.
6,拉卡拉,近日，拉卡拉、汇付天下、百付宝、平安付等支付机构发布公告称已经完成“断直连”工作，也有部分机...,市场相关.公示公告.
7,拉卡拉,近日，拉卡拉与蓝色光标传媒集团联合宣布，成立“昆仑堂大数据研究院”（下文简称“昆仑堂”），汇...,经营相关.经营业绩.区块链.
8,拉卡拉,9月7日，证监会公布的IPO中止审查名单中，拉卡拉支付位列其中，中止原因为申请文件不齐备等导...,市场相关.上市退市.借壳保壳.政策影响.政策影响.公示公告.
9,拉卡拉,近日，拉卡拉联合华为钱包正式推出商家收款产品—华为手机POS，将华为手机变身为商家收款设备，...,经营相关.产品信息.市场相关.上市退市.


In [63]:
for i in range(len(data)):
    sentence=cut_sen(data.loc[i,'NEWS_CONTENT'])
    res=[]
    # loop through every sentence
    for j in sentence:
        if data.loc[i,'COMP_NM'] in j:
            res.append(j)
    data.loc[i,'SEN']='。'.join(res)       


In [64]:
data.loc[0,'SEN']

'\u3000\u3000新华网天津10月27日电（宫碧莹）2020年，位于渤海湾上海河入海口的天津港建设愈发令人瞩目。27日上午，记者在天津港集装箱码头看到，在智能系统指挥下，多辆无人驾驶的集装箱卡车依次行驶在岸边式集装箱装卸桥区域，码头着力打造的“智慧港口”建设已规模彰显。\r\n\u3000\u3000天津港是世界人工深水大港，码头等级达30万吨级，岸线长度40.25公里，拥有集装箱航线130条，每月航班550余班，同世界上200多个国家和地区的800多个港口保持贸易往来。\r\n\r\n\u3000\u3000天津港集装箱码头。今年以来，天津港集团积极应对形式变化，充分发挥港口枢纽“先行官”作用。前三季度，天津港集团累计完成集装箱吞吐量1376.5万标准箱，同比增长5.2%；累计完成货物吞吐量3.36亿吨，同比增长4.3%，呈现出稳中有进、进中向好的发展态势。\r\n\u3000\u3000在绿色港口建设方面，天津港集装箱码头相继实施场桥油改电、场桥自动上电、船舶岸基供电系统、安全监控中心建设，大力推广应用清洁能源和绿色照明，打造环保低碳、环境友好的绿色发展新格局。\r\n\r\n\u3000\u3000天津港集装箱码头物流堆场。新华网 宫碧莹 摄\r\n\u3000\u3000技术赋能 天津港实现码头作业全流程无人化\r\n\u3000\u3000“10月17日，天津港集装箱码头向全球展示了自动化示范区的全无人操作，实现了传统码头全流程无人作业展示。目前，天津港集装箱码头已开启全流程无人化操作新篇章。”天津港集装箱码头有限公司党委书记、董事长刘杰强介绍到。\r\n\r\n\u3000\u3000天津港集装箱码头岸边式集装箱装卸桥无人操作区。新华网 宫碧莹 摄\r\n\u3000\u3000创新突破 集装箱地面解锁站智慧升级\r\n\u3000\u3000天津港集装箱码头通过自主研制集装箱地面智能解锁站，实现车辆自动定位、机械臂加解锁的全自动作业。\r\n\r\n\u3000\u3000天津港集装箱码头智能解锁站。天津港持续建设现代信息技术与港口各领域发展深度融合的步伐铿锵有力，站在新的起点上，天津港未来可期'

In [65]:
for i in range(len(data)):
    data.loc[i, 'Category'] = ''
    
    for k in key:
        for v in d[k]:
            if v in data.loc[i, 'SEN']: 
                data.loc[i, 'Category']+=k
                data.loc[i, 'Category']+='.'
    
    if len(data.loc[i, 'Category'])==0:
        data.loc[i, 'Category']='其他'

data

,ID,COMP_NM,NEWS_TITLE,NEWS_TIME,NEWS_SOURCE,NEWS_CONTENT,THE_URL,SEN,Category
0,1,天津港,【2020·指尖城市】天津港：用技术打造智慧港口新标杆 用智慧赋能港口建设新海蓝,2020-10-28 16:33:16,新华网,新华网天津10月27日电（宫碧莹）2020年，位于渤海湾上海河入海口的天津港建设愈发令人...,http://www.xinhuanet.com/info/2020-10/28/c_139...,新华网天津10月27日电（宫碧莹）2020年，位于渤海湾上海河入海口的天津港建设愈发令人...,环保问题.
1,3,天津港,天津港再添“一带一路”新航线,2020-10-28 10:48:46,新华网,新华社天津10月28日电（记者王井怀）日前，随着“金星佐伊”轮靠泊天津港联盟国际集装箱码...,http://www.xinhuanet.com/fortune/2020-10/28/c_...,新华社天津10月28日电（记者王井怀）日前，随着“金星佐伊”轮靠泊天津港联盟国际集装箱码...,生产产能.
2,6,中国科技出版传媒股份有限公司,镜观中国·新华社国内新闻照片一周精选,2020-11-28 19:20:43,新华网,雪后长城\r\n 11月21日，北京出现降雪，雪后的慕田峪长城银装素裹，云雾缭绕，美如...,http://www.xinhuanet.com/2020-11/28/c_11267981...,,其他
3,7,中国科技出版传媒股份有限公司,金典“夺冠”，安慕希“售罄”！伊利电商双11稳居行业榜首,2020-11-12 16:51:49,新华网,今年双11期间，乳业表现尤为活跃，成为电商销售最火热的品类之一。星图数据显示，11月11...,http://www.xinhuanet.com/food/2020-11/12/c_112...,,其他
4,8,中国科技出版传媒股份有限公司,第三十届中国新闻奖获奖作品目录（348件）,2020-11-02 23:52:17,新华网,新华社北京11月2日电\r\n第三十届中国新闻奖获奖作品目录（348件）\r\n 特别...,http://www.xinhuanet.com/2020-11/02/c_11266892...,,其他
5,10,万华化学,从“华山一条路”走出的“一体化”化学王国——新时代新国企万华改革创新启示录（中）,2019-08-13 10:52:14,经济参考报,万华烟台工业园装置图。资料照片\r\n\r\n 在烟台万华化学集团中央研究院，工作人员在做...,http://www.xinhuanet.com/2019-08/13/c_11248692...,资料照片\r\n\r\n 在烟台万华化学集团中央研究院，工作人员在做聚丙烯材料摆锤冲击试验...,生产产能.产品信息.市场相关.专利.
6,12,新世界,“云”上生活——从世界VR产业大会云峰会品味生活新场景,2020-10-19 20:18:16,新华网,新华社南昌10月19日电（记者郭杰文）19日，2020世界VR产业大会云峰会在江西南昌开...,http://www.xinhuanet.com/2020-10/19/c_11266311...,,其他
7,14,新世界,科普：量子是什么？从“微观世界规律”到人类“新物理革命”,2020-10-19 20:13:15,新华网,新华社合肥10月19日电 题：科普：量子是什么？从“微观世界规律”到人类“新物理革命”\...,http://www.xinhuanet.com/2020-10/19/c_11266311...,,其他
8,17,新世界,新华时评：中国经济正增长为世界注入正能量,2020-10-19 18:04:50,新华网,新华社北京10月19日电 题：中国经济正增长为世界注入正能量\r\n 新华社记者刘红霞...,http://www.xinhuanet.com/fortune/2020-10/19/c_...,,其他
9,19,罗平锌电,被点名通报后 云南罗平锌电已投入1.9亿整改环境隐患,2019-02-21 18:31:42,新华网,新华社昆明2月21日电（记者 白靖利）“再过几个月，这里将变成一个美丽的运动场。”面对现...,http://www.xinhuanet.com/local/2019-02/21/c_11...,”面对现场二十余名媒体记者，罗平锌电股份有限公司副董事长李尤立说。\r\n 2018年6月...,环保问题.警示.管理相关.生产产能.


In [49]:
def ReadTxtName(rootdir):
    lines = []
    with open(rootdir, 'r', encoding='utf-8') as file_to_read:
        while True:
            line = file_to_read.readline()
            if not line:
                break
            line = line.strip('\n')
            lines.append(line)
    return lines

txtpath = 'dict/京东标签.txt'
tags = ReadTxtName(txtpath)
tags

['信用预警',
 '承诺失信',
 '兑付不确定',
 '偿付不确定',
 '债务违约',
 '债券违约',
 '中债隐含评级',
 '信用评级下调',
 '评级展望负面',
 '列入评级观察',
 '推迟评级',
 '责令改正',
 '信披问题',
 '财务预警',
 '财务造假',
 '审计意见',
 '担保预警',
 '资金风险',
 '计提坏账准备',
 '管理预警',
 '高层被查',
 '高管违法',
 '高管无法履职',
 '高管失联',
 '贪污受贿',
 '裁员相关',
 '拖欠薪资',
 '员工罢工',
 '自杀猝死',
 '经营预警',
 '停工停产',
 '生产事故',
 '拖欠货款',
 '偷税漏税',
 '资产出售',
 '诉讼纠纷',
 '股权冻结',
 '破产清算',
 '合作终止',
 '业绩下降',
 '垄断信息',
 '侵权抄袭',
 '环保问题',
 '资金占用',
 '资金挪用',
 '经营失联',
 '经营异常',
 '减资',
 '分立',
 '合并',
 '资产查封',
 '资产扣押',
 '资产冻结',
 '合同纠纷',
 '客户投诉',
 '维权',
 '监管预警',
 '监管关注',
 '监管谈话',
 '警示',
 '公开谴责',
 '通报批评',
 '市场禁入',
 '产品预警',
 '产品召回',
 '产品问题',
 '虚假宣传',
 '项目预警',
 '项目通报',
 '终止项目',
 '无证施工',
 '坍塌事故',
 '其他预警',
 '违法违规',
 '立案调查',
 '市值下降',
 '估值下降',
 '推迟发行',
 '取消发行',
 '爆仓',
 '暴雷事件',
 '中毒事故',
 '管理相关',
 '高管变动',
 '股权激励',
 '员工持股计划',
 '经营相关',
 '经营业绩',
 '战略合作',
 '兼并收购',
 '股权质押',
 '增资募资',
 '投融资',
 '招投标',
 '资产重组',
 '利润分配',
 '接管托管',
 '生产产能',
 '关联交易',
 '产品信息',
 '项目签约',
 '市场相关',
 '增持减持',
 '股份回购',
 '股权转让',
 '新股发行',
 '股价下跌',
 '大宗交易',
 '上市退市',
 '借壳保壳',
 '停复牌',


In [67]:
'新世界' in '新华时评：中国经济正增长为世界注入正能量'

False

In [50]:
# categorize news
cat=pd.read_excel('dict/category.xlsx')

In [60]:
cat

,Key,Value
0,信用预警,信用预警
1,承诺失信,承诺失信
2,兑付/偿付不确定,兑付不确定，偿付不确定
3,债券/债务违约,债务违约，债券违约
4,中债隐含评级,中债隐含评级
...,...,...
114,区块链,区块链
115,竣工投用,竣工投用
116,组织成立,组织成立
117,5G,5G


In [51]:
d={}
key=cat['Key']
value=cat['Value']

In [52]:
value

0             信用预警
1             承诺失信
2      兑付不确定，偿付不确定
3        债务违约，债券违约
4           中债隐含评级
          ...     
114            区块链
115           竣工投用
116           组织成立
117             5G
118           自动驾驶
Name: Value, Length: 119, dtype: object

In [53]:
# create dictionary
for i in range(len(key)):
    d[key[i]]=str(value[i]).split('，')
d

{'信用预警': ['信用预警'],
 '承诺失信': ['承诺失信'],
 '兑付/偿付不确定': ['兑付不确定', '偿付不确定'],
 '债券/债务违约': ['债务违约', '债券违约'],
 '中债隐含评级': ['中债隐含评级'],
 '信用评级下调': ['信用评级下调', '评级下调'],
 '评级展望负面': ['评级展望负面', '评级负面'],
 '列入评级观察': ['列入评级观察', '评级观察'],
 '推迟评级': ['推迟评级'],
 '责令改正': ['责令改正', '责令整改'],
 '信披问题': ['信息披露', '信披问题'],
 '财务预警': ['财务预警'],
 '财务造假': ['财务造假'],
 '审计意见': ['审计意见'],
 '担保预警': ['担保预警'],
 '资金风险': ['资金风险'],
 '计提坏账准备': ['计提坏账'],
 '管理预警': ['管理预警'],
 '高层被查': ['高层被查'],
 '高管违法': ['高管违法'],
 '高管失联/无法履职': ['失联', '无法履职', '失职'],
 '贪污受贿': ['贪污', '受贿', '行贿', '贿赂'],
 '裁员相关': ['裁员'],
 '拖欠薪资': ['拖欠工资', '拖欠'],
 '员工罢工': ['罢工'],
 '自杀猝死': ['自杀', '猝死'],
 '经营预警': ['经营预警', '经营不善'],
 '停工停产': ['停工停产', '停工', '停产'],
 '生产事故': ['生产事故'],
 '拖欠货款': ['拖欠货款'],
 '偷税漏税': ['偷税漏税', '偷税', '漏税'],
 '资产出售': ['资产出售'],
 '诉讼纠纷': ['诉讼纠纷', '诉讼', '纠纷'],
 '股权冻结': ['股权冻结'],
 '破产清算': ['破产清算', '破产', '清算'],
 '合作终止': ['合作终止'],
 '业绩下降': ['业绩下降'],
 '垄断信息': ['垄断信息', '垄断'],
 '侵权抄袭': ['侵权抄袭', ' 侵权', '抄袭'],
 '环保问题': ['环保问题', '环保'],
 '资金挪用/占用': ['资金挪用', '资金占用'],
 '经营失联(

In [7]:
for i in range(len(data)):
    data.loc[i, 'Category'] = ''
    
    for k in key:
        for v in d[k]:
            if v in data.loc[i, 'NEWS_CONTENT']: 
                data.loc[i, 'Category']+=k
                data.loc[i, 'Category']+='.'
    
    if len(data.loc[i, 'Category'])==0:
        data.loc[i, 'Category']='其他'

data

,ID,COMP_NM,NEWS_TITLE,NEWS_TIME,NEWS_SOURCE,NEWS_CONTENT,THE_URL,Category
0,1,西部证券,时隔两年再放行 证监会批准纽银梅隆西部基金公司成立,2010-07-08 08:07,经济观察网,经济观察网 记者 欧阳晓红 时隔两年，中国基金市场终添一新员。7月7日，本网获悉，中国证券监...,http://www.eeo.com.cn/2010/0708/174886.shtml,管理相关.高管变动.投融资.产品信息.市场相关.
1,2,西部证券,西部证券IPO搁浅,2010-03-08 07:03,经济观察网,经济观察报 记者 中彬 李保华 西部证券上市申请的靴子终于落地。\n\r\n\r\n本报独家...,http://www.eeo.com.cn/2010/0308/164497.shtml,减资/分立/合并.管理相关.股权激励.经营业绩.兼并收购.兼并收购.增资募资.投融资.投融资...
2,3,拉卡拉,产业互联网战略成果显现 拉卡拉前三季度营收净利双增长,2020-10-22 19:58,经济观察网,10月22日，拉卡拉发布2020年三季报，数据显示，第三季度营收16.14亿元，同比增长35...,http://www.eeo.com.cn/2020/1022/424674.shtml,经营相关.经营业绩.产品信息.市场相关.上市退市.评级信息.
3,4,拉卡拉,拉卡拉受邀亮相2020中国国际服务贸易交易会,2020-09-05 12:43,经济观察网,9月5日，2020年中国国际服务贸易交易会（以下简称“服贸会”）在中国国家会议中心（北京）举...,http://www.eeo.com.cn/2020/0905/408736.shtml,管理相关.经营相关.产品信息.牌照.会议相关.5G.
4,5,拉卡拉,拉卡拉成功上市 从支付收单到全方位赋能商户 上市之后的拉卡拉 会给市场带来些什么？,2019-04-26 19:23,经济观察网,4月25日，头顶A股“支付第一股”光环的拉卡拉正式登陆深交所创业板，其后续的表现无疑将成为第...,http://www.eeo.com.cn/2019/0426/354516.shtml,破产清算.管理相关.经营相关.产品信息.项目签约.市场相关.上市退市.
5,6,拉卡拉,拉卡拉支付IPO：B端业务占比超89%,2019-03-13 15:03,经济观察网,（图片来源：全景视觉）\r\n经济观察网 记者 黄一帆 几经波折之后，拉卡拉离资本市场终于尚...,http://www.eeo.com.cn/2019/0313/350246.shtml,经营相关.产品信息.市场相关.
6,7,拉卡拉,"拉卡拉、百付宝等多家支付机构已提前完成""备付金100%集中交存""与""断直连""",2019-01-07 14:15,经济观察网,（图片来源：全景视觉）\r\n经济观察网 记者 老盈盈 近日，拉卡拉、汇付天下、百付宝、平安...,http://www.eeo.com.cn/2019/0107/344860.shtml,破产清算.产品信息.市场相关.牌照.公示公告.
7,8,拉卡拉,拉卡拉成立大数据研究院，呈“北有昆仑堂 南有达摩院”态势,2018-04-09 20:30,经济观察网,近日，拉卡拉与蓝色光标传媒集团联合宣布，成立“昆仑堂大数据研究院”（下文简称“昆仑堂”），汇...,http://www.eeo.com.cn/2018/0409/326312.shtml,管理相关.经营相关.经营业绩.区块链.
8,9,拉卡拉,拉卡拉IPO因更换签字律师暂停 同业律师称比较常见,2017-09-12 20:22,经济观察网,（图片来源：全景视觉）\r\n经济观察网 记者 黄一帆 实习生 那诗雨 9月7日，证监会公布...,http://www.eeo.com.cn/2017/0912/312733.shtml,市场相关.增持减持.上市退市.借壳保壳.政策影响.政策影响.公示公告.
9,10,拉卡拉,拉卡拉携手华为钱包发布手机POS 全方位助力商户经营,2020-07-09 11:35,经济观察网,近日，拉卡拉联合华为钱包正式推出商家收款产品—华为手机POS，将华为手机变身为商家收款设备，...,http://www.eeo.com.cn/2020/0709/389162.shtml,经营相关.产品信息.市场相关.上市退市.


## Rename company

In [27]:
# company name
import pandas as pd
name = pd.read_excel('采集企业名录.xlsx')

In [28]:
name.head()

,公司简称,代码,公司全称,Unnamed: 3
0,浦发银行,600000,上海浦东发展银行股份有限公司,NaN
1,白云机场,600004,广州白云国际机场股份有限公司,NaN
2,东风汽车,600006,东风汽车股份有限公司,NaN
3,中国国贸,600007,中国国际贸易中心股份有限公司,NaN
4,首创股份,600008,北京首创股份有限公司,NaN


In [21]:
comp=['天元大数据信用管理有限公司',
      '山东省供销社农产品有限公司',
      '山东福瑞达生物工程有限公司',
      '山东小鸭集团小家电有限公司',
      '山东布克图书有限公司',
      '山东寰宇线缆有限公司',
      '山东高速国储物流有限公司',
      '山东宇索电子科技有限公司',
      '济南跨界电子商务有限公司',
      '济南爱动动漫科技有限公司',
      '济南鲁青置业有限公司',
      '山东丽波电子科技有限公司',
      '山东小炒旺食品有限公司',
      '山东连胜体育产业有限公司',
      '山东三维大豆蛋白有限公司',
      '山东东都食品有限公司',
      '上海浦东发展银行股份有限公司',
      '广州白云国际机场股份有限公司',
      '东风汽车股份有限公司',
      '中国国际贸易中心股份有限公司',
      '北京首创股份有限公司',
      '上海国际机场股份有限公司',
      '内蒙古包钢钢联股份有限公司',
      '华能国际电力股份有限公司',
      '安徽皖通高速公路股份有限公司',
      '华夏银行股份有限公司',
      '中国民生银行股份有限公司',
      '日照港股份有限公司',
      '上海国际港务(集团)股份有限公司',
      '宝山钢铁股份有限公司',
      '河南中原高速公路股份有限公司',
      '上海电力股份有限公司',
      '山东钢铁股份有限公司',
      '浙江浙能电力股份有限公司',
      '华能澜沧江水电股份有限公司',
      '中远海运能源运输股份有限公司',
      '华电国际电力股份有限公司',
      '中国石油化工股份有限公司',
      '中国南方航空股份有限公司',
      '中信证券股份有限公司',
      '三一重工股份有限公司',
      '福建发展高速公路股份有限公司',
      '湖北楚天智能交通股份有限公司']

In [31]:
# 去掉股份有限公司
def rename(comp):
    
    prov=['河北', '山西', '辽宁', '吉林', '黑龙江', '江苏', '浙江', '安徽', '福建', \
      '江西','山东', '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', \
      '陕西', '甘肃', '青海', '台湾', '内蒙古', '广西', '西藏', '宁夏', '新疆',\
      '北京', '天津', '上海', '重庆', '香港', '澳门']
    
    res=[]
    # 先确定是不是上市公司:分类
    for i in range(len(name)):
        if name.loc[i,'公司全称']==comp:
            res.append(name.loc[i,'公司简称'])
            
    # 去后缀
    comp=comp.replace('有限公司', '')
    comp=comp.replace('股份', '')

    if len(comp)>4:
        for j in prov:
            if j in comp:
                # delete keywords
                comp=comp.replace(j,'')
    res.append(comp)
    
    return res

In [32]:
rename('宝山钢铁股份有限公司')

['宝钢股份', '宝山钢铁']

In [33]:
rename('山东福瑞达生物工程有限公司')

['福瑞达生物工程']

In [57]:
j=0
for i in range(len(name)):
    if len(name.loc[i,'公司简称'])<=3:
        j+=1
        # print(name.loc[i,'公司简称'])
        # print(name.loc[i,'简称'])
print(j)

487


## Named-entity recognition

In [8]:
import spacy
import zh_core_web_sm
nlp = zh_core_web_sm.load()

In [17]:
doc = nlp('西门子将努力参与中国的三峡工程建设。')
for token in doc:
  print(token.text,token.pos_,token.tag_)

西门子 PROPN NR
将 ADV AD
努力 ADV AD
参与 VERB VV
中国 PROPN NR
的 PART DEG
三峡 PROPN NR
工程 NOUN NN
建设 NOUN NN
。 PUNCT PU


In [39]:
doc = nlp('中国石化即将破产')
for token in doc:
  print(token.text,token.dep_,token.head)

中国 compound:nn 石化
石化 nsubj 破产
即将 advmod 破产
破产 ROOT 破产


In [40]:
from spacy import displacy
displacy.render(doc,style='dep')

In [9]:
doc = nlp('太平洋将努力参与中国的三峡工程建设，中国石化即将破产。')
for ent in doc.ents:
    print(ent.text,ent.label_)

太平洋 LOC
中国 GPE
三峡工程 EVENT
中国石化 ORG


In [32]:
from spacy import displacy
displacy.render(doc,style='ent')

In [28]:
doc = nlp('字节跳动将努力参与中国的三峡工程建设。')
for token in doc:
  print(token.text,token.pos_,token.tag_)

字节 NOUN NN
跳动 VERB VV
将 ADV AD
努力 ADV AD
参与 VERB VV
中国 PROPN NR
的 PART DEG
三峡 PROPN NR
工程 NOUN NN
建设 NOUN NN
。 PUNCT PU


In [5]:
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch
from spacy.tokens import Doc
from spacy.training import Example

TRAIN_DATA = [('what is the price of polo?', {'entities': [(21, 25, 'PrdName')]}), 
              ('what is the price of ball?', {'entities': [(21, 25, 'PrdName')]}), 
              ('what is the price of jegging?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of t-shirt?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of jeans?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of bat?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of shirt?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of bag?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of cup?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of jug?', {'entities': [(21, 24, 'PrdName')]}), 
              ('what is the price of plate?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of glass?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of moniter?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of desktop?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of bottle?', {'entities': [(21, 27, 'PrdName')]}), 
              ('what is the price of mouse?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of keyboad?', {'entities': [(21, 28, 'PrdName')]}), 
              ('what is the price of chair?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of table?', {'entities': [(21, 26, 'PrdName')]}), 
              ('what is the price of watch?', {'entities': [(21, 26, 'PrdName')]})]


def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
     
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        # ner = nlp.create_pipe('ner')
        ner=nlp.add_pipe('ner', last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            
            batches = minibatch(TRAIN_DATA,size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                examples = []
                for text, annots in batch:
                    examples.append(Example.from_dict(nlp.make_doc(text), annots))
                nlp.update(examples, losses=losses)
            print("Losses", losses)
            
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

#Test your text
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Statring iteration 0
Losses {'ner': 96.62984853982925}
Statring iteration 1
Losses {'ner': 25.39988318865653}
Statring iteration 2
Losses {'ner': 8.019709318536655}
Statring iteration 3
Losses {'ner': 1.727724821297961}
Statring iteration 4
Losses {'ner': 1.3891274837986352}
Statring iteration 5
Losses {'ner': 5.571410102027647}
Statring iteration 6
Losses {'ner': 2.233457066181507}
Statring iteration 7
Losses {'ner': 2.9511732373785}
Statring iteration 8
Losses {'ner': 1.7585950680754818}
Statring iteration 9
Losses {'ner': 2.142415300202229}
Statring iteration 10
Losses {'ner': 0.3383257065414231}
Statring iteration 11
Losses {'ner': 0.12205734401274566}
Statring iteration 12
Losses {'ner': 0.009236358735382911}
Statring iteration 13
Losses {'ner': 0.00021850403070871932}
Statring iteration 14
Losses {'ner': 0.00011764531484482899}
Statring iteration 15
Losses {'ner': 7.705900778380006e-06}
Statring iteration 16
Losses {'ner': 6.932176126678062e-07}
Statring iteration 17
Losses {'ner

KeyboardInterrupt: Interrupted by user

In [13]:
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch
from spacy.tokens import Doc
from spacy.training import Example

TRAIN_DATA = [('西门子利用建设银行提供的借口获取数据', {'entities': [(0,3, 'PrdName')]}), 
              ('广州银行参与建设三峡大坝', {'entities': [(0, 4, 'PrdName')]}), 
              ('工商银行参与建设三峡大坝', {'entities': [(0, 4, 'PrdName')]}), 
              ('齐鲁银行参与建设三峡大坝', {'entities': [(0, 4, 'PrdName')]}), 
              ('北京银行参与建设三峡大坝', {'entities': [(0, 4, 'PrdName')]}), 
              ('西部证券参与建设三峡大坝', {'entities': [(0, 4, 'PrdName')]})
]


def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('zh')  # create blank Language class
     
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        # ner = nlp.create_pipe('ner')
        ner=nlp.add_pipe('ner', last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            
            batches = minibatch(TRAIN_DATA,size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                examples = []
                for text, annots in batch:
                    examples.append(Example.from_dict(nlp.make_doc(text), annots))
                nlp.update(examples, losses=losses)
            print("Losses", losses)
            
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)

# Save our trained Model
# modelfile = input("Enter your Model Name: ")
# prdnlp.to_disk(modelfile)

#Test your text
test_text = input("Enter your testing text: ")
doc = prdnlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Statring iteration 0
Losses {'ner': 64.15424156188965}
Statring iteration 1
Losses {'ner': 55.1140598654747}
Statring iteration 2
Losses {'ner': 35.76620641350746}
Statring iteration 3
Losses {'ner': 12.719263942912221}
Statring iteration 4
Losses {'ner': 7.260201413431787}
Statring iteration 5
Losses {'ner': 5.596546223605856}
Statring iteration 6
Losses {'ner': 89.80663683757993}
Statring iteration 7
Losses {'ner': 118.51108086109161}
Statring iteration 8
Losses {'ner': 98.15313843531044}
Statring iteration 9
Losses {'ner': 17.799040705828702}
Statring iteration 10
Losses {'ner': 13.350520857613446}
Statring iteration 11
Losses {'ner': 9.476065351962461}
Statring iteration 12
Losses {'ner': 5.322022196774924}
Statring iteration 13
Losses {'ner': 2.7966761269308336}
Statring iteration 14
Losses {'ner': 1.606933721520818}
Statring iteration 15
Losses {'ner': 0.35673417799810636}
Statring iteration 16
Losses {'ner': 0.011360147396796805}
Statring iteration 17
Losses {'ner': 0.0001097332